Demonstration of software with setup from Borregaard et.al 2015

In [1]:
# install some (non-mainstream) packages that are being used by software
!pip install qutip 
!pip install type_enforced

import src as ASQOS # Abstract Symbolic python package for Quantum Optical Systems

In [2]:
from sympy import Symbol

#Borregaard Test
DE = Symbol("Delta_E",real=True)
De = Symbol("Delta_e",real=True)
gamma_f = Symbol("gamma_f",positive= True)
gamma_g = Symbol("gamma_g",positive= True)
gamma = Symbol('gamma', positive = True)
kappa_c = Symbol('kappa_c', positive = True)
g = Symbol('g',real=True)
g_f = Symbol('g_f',real=True)
Omega = Symbol('Omega',real=True)

component_list = []

# Cavity Definitiom
## Energy Levels  
zero_ph = ASQOS.EnergyLevel('0ph', energy = 0)
one_ph = ASQOS.EnergyLevel('1ph', energy = 0)
## Component
cavity = ASQOS.Component([zero_ph, one_ph], 'cavity')
## Component Interactions
ASQOS.Decay(one_ph , zero_ph, kappa_c)
emission_into_cavity = ASQOS.Transition(one_ph , zero_ph)
## append to list of components
component_list.append(cavity)

for i in range(2):
    # Qubits Definitiom
    ## Energy Levels  
    zero = ASQOS.EnergyLevel(f'0_{i}',0)
    one = ASQOS.EnergyLevel(f'1_{i}',0)
    e = ASQOS.EnergyLevel(f'e_{i}', De)
    dump = ASQOS.EnergyLevel(f'o_{i}',0)
    ## Component
    qubit_atom = ASQOS.Component([zero,one,e,dump], f'qubit_{i}')
    ## Component Interactions
    e_to_one_transition = ASQOS.Transition(one,e)
    ASQOS.Coupling(e_to_one_transition,emission_into_cavity, g)
    ASQOS.Decay(e,dump, gamma)
    ## append to list of components
    component_list.append(qubit_atom)

# Auxiliary atom Definitiom
## Energy Levels 
g_state = ASQOS.EnergyLevel(f'g',0)
f_state = ASQOS.EnergyLevel(f'f',0)
E = ASQOS.EnergyLevel(f'E', DE)
## Component
aux_atom = ASQOS.Component([g_state,f_state,E], f'aux_atom')
## Component Interactions
E_to_f_transition = ASQOS.Transition(f_state,E)
ASQOS.Coupling(E_to_f_transition,emission_into_cavity, g_f)
ASQOS.Rabi(E,g_state,Omega)
ASQOS.Decay(E,g_state,gamma_g)
ASQOS.Decay(E,f_state,gamma_f)

component_list.append(aux_atom)

In [3]:
s = ASQOS.QOpsSystem('Bor2015_2q', component_list)

In [4]:
# Compile system so that all components "know" about other components
s.compile()

In [6]:
# Run the effective operator formalism routine 
s.obtain_effective_operators()

Constructing states and excitations ...
Constructing ground and first-excited statespace ...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian ...
Inverting NJ_hamiltonian ...
Constructing eff_hamiltonian and effective lindblad operators ...

QOpsSystem Bor2015_2q effective operators in 9.2 seconds.


In [7]:
# View effective Hamiltonian
# 8 dimensions instead of 2 because f is taken into consideration as ground state
s.effective_operator_formalism.eff_hamiltonian_gs

Matrix([
[-0.5*Omega**2*(0.5*I*kappa_c/(-1.0*g_f**2 + 0.5*I*kappa_c*(1.0*Delta_E + 0.5*I*(gamma_f + gamma_g))) - 0.5*I*kappa_c/(-1.0*g_f**2 - 0.5*I*kappa_c*(1.0*Delta_E - 0.5*I*(gamma_f + gamma_g)))), 0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           0, 0,                                             

In [9]:
# View corresponding states
s.effective_operator_formalism.gs_states

array(['|0ph|0_0|0_1|g|', '|0ph|0_0|0_1|f|', '|0ph|0_0|1_1|g|',
       '|0ph|0_0|1_1|f|', '|0ph|1_0|0_1|g|', '|0ph|1_0|0_1|f|',
       '|0ph|1_0|1_1|g|', '|0ph|1_0|1_1|f|'], dtype='<U15')

In [10]:
# View also one of the lindblad operators
s.effective_operator_formalism.eff_lindblad_list[0]

Matrix([
[                                                                                                   0, 0, 0,                                                                                                                                                                                                                                             0, 0, 0, 0, 0, 0, 0,                                                                                                                                                                                                                                             0, 0, 0,                                                                                                                                                                                                                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-1.0*Omega*g_f*sqrt(kappa_c)/(-1.0*g_f**2 - 0.5*I*kappa_c*(1.0*Delta_E - 